<a href="https://colab.research.google.com/github/gzcarlos/dez2024-ws1/blob/main/Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [10]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [15]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [ ]:
#Install the dependencies
#%%capture
!pip install dlt[duckdb]

### Question 1: What is the sum of the outputs of the generator for limit = 5?

In [12]:
sum_outputs = 0
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    sum_outputs += sqrt_value
    print(sqrt_value)

print(f'Sum of the outputs of the generator when limit = 5: {sum_outputs}')

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
Sum of the outputs of the generator when limit = 5: 8.382332347441762


### Question 2: What is the 13th number yielded by the generator?

In [14]:
limit = 13
generator = square_root_generator(limit)
iter_id = 1
for sqrt_value in generator:
    if iter_id == 13:
      print(sqrt_value)
    iter_id += 1

3.605551275463989


### Question 3: Append the 2 generators. After correctly appending the data, calculate the sum of all ages of people.

##### Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.

In [23]:
import dlt
import duckdb
# create the generator
p_generator_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
# run the data pipeline to save the data
info = p_generator_pipeline.run(
    people_1(),
	table_name="people",
	write_disposition="replace"
)

# show process
# display(info)

# get the data and sum the ages as output
conn = duckdb.connect(f"{p_generator_pipeline.pipeline_name}.duckdb")
age_sum = conn.sql("SELECT sum(Age) as age_sum FROM generators.people").df()
display(age_sum)

,age_sum
0,140.0


##### Append the second generator to the same table as the first.

In [24]:
# append second
info = p_generator_pipeline.run(
    people_2(),
	table_name="people",
	write_disposition="append"
)
# show process
display(info)



LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x7f3b0ef089d0>, metrics={'1708180071.3484526': [{'started_at': DateTime(2024, 2, 17, 14, 27, 51, 717914, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 17, 14, 27, 51, 955523, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////workspaces/dez2024-ws1/dlt_ipykernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='generators', loads_ids=['1708180071.3484526'], load_packages=[LoadPackageInfo(load_id='1708180071.3484526', package_path='/home/codespace/.dlt/pipelines/dlt_ipykernel_launcher/load/loaded/1708180071.3484526', state='loaded', schema=Schema dlt_ipykernel_launcher at 139891630829776, schema_update={'people': {'name': 'people', 'columns': {'occupation': {'name': 'occupation', 'data_type': 'text', 'nullable': True}}

##### After correctly appending the data, calculate the sum of all ages of people.

In [25]:
# get the sum now with both data loaded
conn = duckdb.connect(f"{p_generator_pipeline.pipeline_name}.duckdb")
age_sum_total = conn.sql("SELECT sum(Age) as age_sum FROM generators.people").df()
display(age_sum_total)

,age_sum
0,353.0


### Question 4. Merge the 2 generators using the ID column. Calculate the sum of ages of all the people loaded as described above.

##### A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.
##### Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.
##### After loading, you should have a total of 8 records, and ID 3 should have age 33.

In [27]:
import dlt
import duckdb
# run the data pipeline to save the data
info = p_generator_pipeline.run(
    people_1(),
	table_name="people_data",
	write_disposition="replace",
    primary_key='ID'
)
# show info
# display(info)
# merge the second generator in the data with the same id
info = p_generator_pipeline.run(
    people_2(),
	table_name="people_data",
	write_disposition="merge",
    primary_key='ID'
)
# show info
# display(info)
# get all the records loaded
conn = duckdb.connect(f"{p_generator_pipeline.pipeline_name}.duckdb")
print('All People loaded')
people = conn.sql("SELECT * FROM generators.people_data").df()
display(people)

All People loaded


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708181511.5523071,LIOkCo32U4ZSRA,None
1,2,Person_2,27,City_A,1708181511.5523071,RPOX+SJAgPs7zw,None
2,8,Person_8,38,City_B,1708181512.2276635,xCYEIHRl+TsJGw,Job_8
3,5,Person_5,35,City_B,1708181512.2276635,x+rQxr8cRgv+yQ,Job_5
4,7,Person_7,37,City_B,1708181512.2276635,wTNcpvyZ6fR8UQ,Job_7
5,4,Person_4,34,City_B,1708181512.2276635,0V0Y9m7L2he3Jw,Job_4
6,3,Person_3,33,City_B,1708181512.2276635,tSpB5tCwHOnciQ,Job_3
7,6,Person_6,36,City_B,1708181512.2276635,l+vvlCcmGWneCg,Job_6


##### Calculate the sum of ages of all the people loaded as described above.

In [28]:
print('All People loaded')
sum_people_age = conn.sql("SELECT sum(Age) as sum_people_age FROM generators.people_data").df()
display(sum_people_age)

All People loaded


,sum_people_age
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX